<a target="_blank" href="https://colab.research.google.com/github/urcraft/llm_lecture_notebooks/blob/main/06_Gemini_API_RAG_InMemory_VectorDB.ipynb">   <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/> </a>

# Gemini API + In-Memory Vector DB (RAG)

## What you will build
- A minimal Retrieval-Augmented Generation (RAG) pipeline.
- Gemini embeddings for chunked documents.
- Chroma in-memory vector database for retrieval.
- Gemini answer generation grounded in retrieved context.

Expected runtime: 10-20 minutes
Expected cost: Zero for Gemini Free tier API usage


## Online References Used
- Gemini embeddings API reference: https://ai.google.dev/api/embeddings
- Gemini text generation guide: https://ai.google.dev/gemini-api/docs/system-instructions
- Chroma in-memory client docs: https://docs.trychroma.com/docs/run-chroma/clients
- Chroma Python reference: https://docs.trychroma.com/reference/python
- Gemini cookbook repository: https://github.com/google-gemini/cookbook


In [1]:
# Force-install compatible versions for the Colab environment
%pip install -q -U google-genai chromadb \
    "pandas==2.2.2" \
    "requests==2.32.4" \
    "opentelemetry-sdk==1.22.0" \
    "opentelemetry-exporter-otlp-proto-http==1.22.0"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.8/728.8 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6

In [2]:
import os
import re
import textwrap
import requests
import pandas as pd
import chromadb

from google import genai


In [4]:
# Set your API key in environment variable GOOGLE_API_KEY before running.
# In Colab, you can also use a secret named GOOGLE_API_KEY.
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    try:
        from google.colab import userdata
        api_key = userdata.get("GOOGLE_API_KEY")
    except Exception:
        api_key = None

if not api_key:
    raise ValueError("Set GOOGLE_API_KEY (environment variable or Colab secret).")

client = genai.Client(api_key=api_key)

EMBED_MODEL = "gemini-embedding-001"
GEN_MODEL = "gemini-3-flash-preview"

print("Gemini client ready")
print("Embedding model:", EMBED_MODEL)
print("Generation model:", GEN_MODEL)


Gemini client ready
Embedding model: gemini-embedding-001
Generation model: gemini-3-flash-preview


In [5]:
# A small, thematically coherent corpus for RAG:
# public docs/README files about Gemini, vector DBs, and RAG tooling.
DOC_URLS = {
    "gemini_cookbook": "https://raw.githubusercontent.com/google-gemini/cookbook/main/README.md",
    "chroma_readme": "https://raw.githubusercontent.com/chroma-core/chroma/main/README.md",
    "langchain_readme": "https://raw.githubusercontent.com/langchain-ai/langchain/master/README.md",
    "llamaindex_readme": "https://raw.githubusercontent.com/run-llama/llama_index/main/README.md"
}

def fetch_text(url: str) -> str:
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.text

raw_docs = {}
for name, url in DOC_URLS.items():
    raw_docs[name] = fetch_text(url)

print("Downloaded documents:", list(raw_docs.keys()))
for k, v in raw_docs.items():
    print(f"- {k}: {len(v):,} chars")


Downloaded documents: ['gemini_cookbook', 'chroma_readme', 'langchain_readme', 'llamaindex_readme']
- gemini_cookbook: 14,088 chars
- chroma_readme: 5,475 chars
- langchain_readme: 6,867 chars
- llamaindex_readme: 13,291 chars


In [6]:
def clean_text(text: str) -> str:
    text = text.replace("\r", "")
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def chunk_text(text: str, chunk_size: int = 1200, overlap: int = 150):
    text = clean_text(text)
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        if end == len(text):
            break
        start = end - overlap
    return chunks

records = []
for doc_id, text in raw_docs.items():
    chunks = chunk_text(text)
    for i, chunk in enumerate(chunks):
        records.append({
            "id": f"{doc_id}_chunk_{i}",
            "doc_id": doc_id,
            "chunk_index": i,
            "text": chunk,
            "source_url": DOC_URLS[doc_id],
        })

df_chunks = pd.DataFrame(records)
print("Total chunks:", len(df_chunks))
df_chunks.head(3)


Total chunks: 40


,id,doc_id,chunk_index,text,source_url
0,gemini_cookbook_chunk_0,gemini_cookbook,0,# Welcome to the Gemini API Cookbook\n\nThis c...,https://raw.githubusercontent.com/google-gemin...
1,gemini_cookbook_chunk_1,gemini_cookbook,1,ation).\n> \n> **🍌 Nano-Banana Pro**: Go banan...,https://raw.githubusercontent.com/google-gemin...
2,gemini_cookbook_chunk_2,gemini_cookbook,2,Practical use cases demonstrating how to comb...,https://raw.githubusercontent.com/google-gemin...


In [7]:
def extract_embeddings(resp):
    # Handles both single and batch embedding response shapes.
    if hasattr(resp, "embeddings") and resp.embeddings is not None:
        out = []
        for emb in resp.embeddings:
            if hasattr(emb, "values"):
                out.append(list(emb.values))
            elif isinstance(emb, dict) and "values" in emb:
                out.append(list(emb["values"]))
        return out

    if hasattr(resp, "embedding") and resp.embedding is not None:
        emb = resp.embedding
        if hasattr(emb, "values"):
            return [list(emb.values)]
        if isinstance(emb, dict) and "values" in emb:
            return [list(emb["values"])]

    raise ValueError("Unexpected embedding response format")

texts = df_chunks["text"].tolist()

try:
    embed_resp = client.models.embed_content(
        model=EMBED_MODEL,
        contents=texts,
        config={"task_type": "RETRIEVAL_DOCUMENT"},
    )
except Exception:
    embed_resp = client.models.embed_content(
        model=EMBED_MODEL,
        contents=texts,
    )

chunk_embeddings = extract_embeddings(embed_resp)

print("Embedding count:", len(chunk_embeddings))
print("Embedding dimension:", len(chunk_embeddings[0]))


Embedding count: 40
Embedding dimension: 3072


In [8]:
# In-memory vector database (temporary).
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="gemini_rag_demo")

collection.add(
    ids=df_chunks["id"].tolist(),
    documents=df_chunks["text"].tolist(),
    embeddings=chunk_embeddings,
    metadatas=[
        {
            "doc_id": row.doc_id,
            "chunk_index": int(row.chunk_index),
            "source_url": row.source_url,
        }
        for row in df_chunks.itertuples(index=False)
    ],
)

print("Stored chunks in Chroma:", collection.count())


Stored chunks in Chroma: 40


In [9]:
def retrieve(query: str, k: int = 8, n_candidates: int = 24, max_per_doc: int = 2):
    try:
        q_resp = client.models.embed_content(
            model=EMBED_MODEL,
            contents=[query],
            config={"task_type": "RETRIEVAL_QUERY"},
        )
    except Exception:
        q_resp = client.models.embed_content(
            model=EMBED_MODEL,
            contents=[query],
        )

    q_emb = extract_embeddings(q_resp)[0]

    res = collection.query(
        query_embeddings=[q_emb],
        n_results=n_candidates,
    )

    ranked = []
    for i in range(len(res["ids"][0])):
        ranked.append({
            "id": res["ids"][0][i],
            "document": res["documents"][0][i],
            "metadata": res["metadatas"][0][i],
            "distance": res["distances"][0][i],
        })

    # Keep top relevance while preventing one source from consuming all slots.
    selected = []
    per_doc = {}
    for r in ranked:
        doc_id = r["metadata"]["doc_id"]
        count = per_doc.get(doc_id, 0)
        if count >= max_per_doc:
            continue
        selected.append(r)
        per_doc[doc_id] = count + 1
        if len(selected) >= k:
            break

    # Fallback: if diversity cap is too strict, top up with best remaining chunks.
    if len(selected) < k:
        selected_ids = {x["id"] for x in selected}
        for r in ranked:
            if r["id"] in selected_ids:
                continue
            selected.append(r)
            if len(selected) >= k:
                break

    return selected


def answer_with_rag(question: str, k: int = 8):
    hits = retrieve(question, k=k)

    context_blocks = []
    citations = []
    for idx, h in enumerate(hits, start=1):
        md = h["metadata"]
        context_blocks.append(
            f"[Context {idx}] source={md['doc_id']} chunk={md['chunk_index']}\n{h['document']}"
        )
        citations.append((idx, md["source_url"]))

    context = "\n\n".join(context_blocks)

    prompt = f"""
You are answering using only the provided context.
If the answer is not supported by context, say: "I don't know based on the provided documents."
Keep the answer concise and include citation markers like [1], [2] tied to context blocks.

Question:
{question}

Context:
{context}
""".strip()

    resp = client.models.generate_content(
        model=GEN_MODEL,
        contents=prompt,
    )

    return {
        "question": question,
        "answer": resp.text,
        "hits": hits,
        "citations": citations,
    }

# Try a question.
result = answer_with_rag("How do Gemini cookbook and Chroma differ in purpose, and where does LangChain fit?")
print("Answer:\n")
print(result["answer"])

print("\nCitation map:")
for idx, url in result["citations"]:
    print(f"[{idx}] {url}")


Answer:

The **Gemini cookbook** is a collection of educational guides and practical examples for building with the Gemini API, covering features like grounding, batch processing, and 3D spatial understanding [1], [2], [7]. In contrast, **Chroma** is a vector database used to store and query embeddings, allowing users to find relevant documents through natural language and compose them into an LLM's context window [3], [4].

**LangChain** fits as an orchestration framework and integration layer that:
*   Connects LLMs to diverse data sources and vector stores like Chroma for real-time data augmentation [4], [5].
*   Provides a modular architecture for model interoperability, rapid prototyping, and agent orchestration (via LangGraph) [5], [6].
*   Is used within Gemini-specific applications to power backend agents [8].

Citation map:
[1] https://raw.githubusercontent.com/google-gemini/cookbook/main/README.md
[2] https://raw.githubusercontent.com/google-gemini/cookbook/main/README.md
[3]